In [1]:
"""
Installs the `arxiv` Python library, which provides a simple interface for accessing the arXiv.org API.

The `arxiv` library allows you to search for and retrieve metadata about research papers published on arXiv.org, a popular open-access repository for scientific papers.
"""
!pip install arxiv

In [2]:
"""
faiss-cpu: Fast similarity search and clustering for large datasets.
langchain: Framework for building LLM-powered applications.
langchain_community: Community-driven extensions for LangChain.
langchain_openai: Using OpenAI LLMs with LangChain.
openai: Accessing OpenAI's API for various machine learning models.
"""

!pip install wikipedia faiss-cpu langchain langchain_community langchain_openai openai

In [3]:
"""
Loads the OpenAI API key from the environment and configures the ChatOpenAI language model with the specified parameters.

The `load_dotenv()` function loads environment variables from a `.env` file, which is used to store sensitive API keys and other configuration values. The `os.environ['OPENAI_API_KEY']` line sets the `OPENAI_API_KEY` environment variable to the value retrieved from the `.env` file.

The `ChatOpenAI` class from the `langchain_openai` module is then used to create an instance of the OpenAI language model, with the specified model name and temperature parameter. This instance can be used to generate text using the OpenAI API.
"""
from dotenv import load_dotenv
import getpass
load_dotenv()
import os
os.environ['OPENAI_API_KEY']=getpass.getpass()
from langchain_openai import ChatOpenAI

llm = ChatOpenAI(model="gpt-3.5-turbo-0125", temperature=0)

In [4]:
"""
This module provides two utility classes for interacting with Wikipedia:

`WikipediaQueryRun`: A class that allows you to run a Wikipedia query and retrieve the results.
`WikipediaAPIWrapper`: A wrapper around the Wikipedia API that provides a more user-friendly interface for querying and retrieving information from Wikipedia.

These utilities can be used to incorporate Wikipedia-based functionality into your applications.
"""
from langchain_community.tools import WikipediaQueryRun
from langchain_community.utilities import WikipediaAPIWrapper

In [5]:
"""
Initializes a WikipediaAPIWrapper instance with a maximum of 1 top result and a maximum of 200 characters per document content.
Then, creates a WikipediaQueryRun instance using the configured WikipediaAPIWrapper.
"""
api_wrapper=WikipediaAPIWrapper(top_k_results=1,doc_content_chars_max=200)
wiki=WikipediaQueryRun(api_wrapper=api_wrapper)

In [6]:
wiki.name

'wikipedia'

In [7]:
"""
This code sets up a document retrieval system using the LangChain library. It loads web-based documents from a specified URL, splits the documents into smaller chunks, and creates a FAISS vector store using OpenAI embeddings. The resulting retriever can be used to search and retrieve relevant documents based on user queries.

The key steps are:
1. Load web-based documents using the WebBaseLoader
2. Split the documents into smaller chunks using the RecursiveCharacterTextSplitter
3. Create a FAISS vector store from the document chunks using OpenAI embeddings
4. Obtain a retriever from the vector store that can be used to search and retrieve relevant documents
"""
from langchain_community.document_loaders import WebBaseLoader
from langchain_community.vectorstores import FAISS
from langchain_openai import OpenAIEmbeddings
from langchain_text_splitters import RecursiveCharacterTextSplitter

loader=WebBaseLoader("https://docs.smith.langchain.com/")
docs=loader.load()
documents=RecursiveCharacterTextSplitter(chunk_size=1000,chunk_overlap=200).split_documents(docs)
vectordb=FAISS.from_documents(documents,OpenAIEmbeddings())
retriever=vectordb.as_retriever()
retriever


USER_AGENT environment variable not set, consider setting it to identify your requests.


VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vectorstore=<langchain_community.vectorstores.faiss.FAISS object at 0x116956c90>)

In [8]:
"""
Creates a LangChain tool that allows searching for information about LangSmith.

This tool should be used for any questions about LangSmith, as it provides a consistent and reliable way to search for relevant information.

Args:
    retriever (Retriever): The retriever to use for searching.
    name (str): The name of the tool.
    description (str): A description of the tool's purpose.

Returns:
    Tool: A LangChain tool that can be used to search for information about LangSmith.
"""
from langchain.tools.retriever import create_retriever_tool
retriever_tool=create_retriever_tool(retriever,"langsmith_search",
                      "Search for information about LangSmith. For any questions about LangSmith, you must use this tool!")

In [9]:
retriever_tool.name

'langsmith_search'

In [10]:
"""
Provides an ArxivAPIWrapper and ArxivQueryRun tool for interacting with the Arxiv API.

The ArxivAPIWrapper is a wrapper around the Arxiv API that allows for querying the API and returning a limited number of results with a maximum character count for the content.

The ArxivQueryRun tool uses the ArxivAPIWrapper to run a query against the Arxiv API and return the results.
"""
## Arxiv Tool
from langchain_community.utilities import ArxivAPIWrapper
from langchain_community.tools import ArxivQueryRun

arxiv_wrapper=ArxivAPIWrapper(top_k_results=1, doc_content_chars_max=200)
arxiv=ArxivQueryRun(api_wrapper=arxiv_wrapper)
arxiv.name

'arxiv'

In [11]:
tools=[wiki,arxiv,retriever_tool]

In [14]:
tools

[WikipediaQueryRun(api_wrapper=WikipediaAPIWrapper(wiki_client=<module 'wikipedia' from '/Users/kranthivardhankurumindla/Desktop/GENAI_BOOTCAMP/.venv/lib/python3.11/site-packages/wikipedia/__init__.py'>, top_k_results=1, lang='en', load_all_available_meta=False, doc_content_chars_max=200)),
 ArxivQueryRun(api_wrapper=ArxivAPIWrapper(arxiv_search=<class 'arxiv.Search'>, arxiv_exceptions=(<class 'arxiv.ArxivError'>, <class 'arxiv.UnexpectedEmptyPageError'>, <class 'arxiv.HTTPError'>), top_k_results=1, ARXIV_MAX_QUERY_LENGTH=300, continue_on_failure=False, load_max_docs=100, load_all_available_meta=False, doc_content_chars_max=200)),
 Tool(name='langsmith_search', description='Search for information about LangSmith. For any questions about LangSmith, you must use this tool!', args_schema=<class 'langchain_core.tools.retriever.RetrieverInput'>, func=functools.partial(<function _get_relevant_documents at 0x109c4d9e0>, retriever=VectorStoreRetriever(tags=['FAISS', 'OpenAIEmbeddings'], vector

In [12]:
"""
Retrieves a prompt from the LangChain hub, which is a collection of pre-built prompts and agents for language models.

The `hub.pull()` function is used to fetch the prompt with the identifier "hwchase17/openai-functions-agent". This prompt can be used as a starting point for building an agent that interacts with OpenAI's language model functions.

The retrieved prompt is stored in the `prompt` variable, which can then be used to configure and customize the agent's behavior.
"""
from langchain import hub
# Get the prompt to use - you can modify this!
prompt = hub.pull("hwchase17/openai-functions-agent")
prompt.messages

/Users/kranthivardhankurumindla/Desktop/GENAI_BOOTCAMP/.venv/lib/python3.11/site-packages/langsmith/client.py:312: LangSmithMissingAPIKeyWarning: API key must be provided when using hosted LangSmith API
  warnings.warn(
/Users/kranthivardhankurumindla/Desktop/GENAI_BOOTCAMP/.venv/lib/python3.11/site-packages/langsmith/client.py:5519: LangChainBetaWarning: The function `loads` is in beta. It is actively being worked on, so the API may change.
  prompt = loads(json.dumps(prompt_object.manifest))


[SystemMessagePromptTemplate(prompt=PromptTemplate(input_variables=[], template='You are a helpful assistant')),
 MessagesPlaceholder(variable_name='chat_history', optional=True),
 HumanMessagePromptTemplate(prompt=PromptTemplate(input_variables=['input'], template='{input}')),
 MessagesPlaceholder(variable_name='agent_scratchpad')]

In [13]:
"""
Creates an OpenAI tools agent using the provided language model (llm), tools, and prompt.

The agent is a powerful tool that can leverage various external tools and services to accomplish complex tasks. It uses the provided language model to understand natural language instructions, and then selects and invokes the appropriate tools to carry out those instructions.

Args:
    llm (LLMChain): The language model to use for the agent.
    tools (list): A list of tools that the agent can use to accomplish tasks.
    prompt (PromptTemplate): The prompt template to use for the agent.

Returns:
    Agent: The created OpenAI tools agent.
"""
### Agents
from langchain.agents import create_openai_tools_agent
agent=create_openai_tools_agent(llm,tools,prompt)

In [14]:
## Agent Executer
from langchain.agents import AgentExecutor
agent_executor=AgentExecutor(agent=agent,tools=tools,verbose=True)
agent_executor

AgentExecutor(verbose=True, agent=RunnableMultiActionAgent(runnable=RunnableAssign(mapper={
  agent_scratchpad: RunnableLambda(lambda x: format_to_openai_tool_messages(x['intermediate_steps']))
})
| ChatPromptTemplate(input_variables=['agent_scratchpad', 'input'], optional_variables=['chat_history'], input_types={'chat_history': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]], 'agent_scratchpad': typing.List[typing.Union[langchain_core.messages.ai.AIMessage, langchain_core.messages.human.HumanMessage, langchain_core.messages.chat.ChatMessage, langchain_core.messages.system.SystemMessage, langchain_core.messages.function.FunctionMessage, langchain_core.messages.tool.ToolMessage]]}, partial_variables={'chat_history': []}, metadata={'lc_hub_owner': 'hwc

In [15]:
agent_executor.invoke({"input":"Tell me about Langsmith"})



> Entering new AgentExecutor chain...

Invoking: `langsmith_search` with `{'query': 'Langsmith'}`


Get started with LangSmith | 🦜️🛠️ LangSmith

Skip to main contentGo to API DocsSearchRegionUSEUGo to AppQuick startTutorialsHow-to guidesConceptsReferencePricingSelf-hostingLangGraph CloudQuick startOn this pageGet started with LangSmithLangSmith is a platform for building production-grade LLM applications. It allows you to closely monitor and evaluate your application, so you can ship quickly and with confidence. Use of LangChain is not necessary - LangSmith works on its own!1. Install LangSmith​PythonTypeScriptpip install -U langsmithyarn add langsmith2. Create an API key​To create an API key head to the Settings page. Then click Create API Key.3. Set up your environment​Shellexport LANGCHAIN_TRACING_V2=trueexport LANGCHAIN_API_KEY=<your-api-key># The below examples use the OpenAI API, though it's not necessary in generalexport OPENAI_API_KEY=<your-openai-api-key>4. Log your first tr

{'input': 'Tell me about Langsmith',
 'output': 'LangSmith is a platform for building production-grade LLM (Large Language Model) applications. It allows you to closely monitor and evaluate your application, enabling you to ship quickly and with confidence. LangSmith can be used independently without the need for LangChain. Here are the steps to get started with LangSmith:\n\n1. Install LangSmith:\n   - Python: `pip install -U langsmith`\n   - TypeScript: `yarn add langsmith`\n\n2. Create an API key:\n   - Head to the Settings page and click on "Create API Key."\n\n3. Set up your environment:\n   - Set environment variables:\n     ```\n     export LANGCHAIN_TRACING_V2=true\n     export LANGCHAIN_API_KEY=<your-api-key>\n     export OPENAI_API_KEY=<your-openai-api-key> (optional)\n     ```\n\n4. Log your first trace:\n   - Tracing to LangSmith for LangChain users does not require using the LangSmith SDK directly if your application is built entirely on LangChain.\n\nLangSmith provides mu

In [16]:
agent_executor.invoke({"input":"What's the paper 1605.08386 about?"})



> Entering new AgentExecutor chain...

Invoking: `arxiv` with `{'query': '1605.08386'}`


Published: 2016-05-26
Title: Heat-bath random walks with Markov bases
Authors: Caprice Stanley, Tobias Windisch
Summary: Graphs on lattice points are studied whose edges come from a finite set of
alloThe paper with identifier 1605.08386 is titled "Heat-bath random walks with Markov bases" by Caprice Stanley and Tobias Windisch. It discusses the study of graphs on lattice points where the edges are derived from a finite set of allocations.

> Finished chain.


{'input': "What's the paper 1605.08386 about?",
 'output': 'The paper with identifier 1605.08386 is titled "Heat-bath random walks with Markov bases" by Caprice Stanley and Tobias Windisch. It discusses the study of graphs on lattice points where the edges are derived from a finite set of allocations.'}

In [18]:
agent_executor.invoke({"input":"tell me about virat kohli"})



> Entering new AgentExecutor chain...

Invoking: `wikipedia` with `{'query': 'Virat Kohli'}`


Page: Virat Kohli
Summary: Virat Kohli (Hindi pronunciation: [ʋɪˈɾɑːʈ ˈkoːɦli] ; born 5 November 1988) is an Indian international cricketer who plays Test and ODI cricket for the Indian national team.Virat Kohli is an Indian international cricketer who plays Test and ODI cricket for the Indian national team. He was born on 5 November 1988. If you would like to know more details about him, feel free to ask!

> Finished chain.


{'input': 'tell me about virat kohli',
 'output': 'Virat Kohli is an Indian international cricketer who plays Test and ODI cricket for the Indian national team. He was born on 5 November 1988. If you would like to know more details about him, feel free to ask!'}